# Module 05 — Semiconductor Zoo
# Notebook 00: Optoelectronics
## LEDs, Photodiodes, Solar Cells, Laser Diodes

---

Semiconductors do not just switch and amplify electrical signals. When photons meet
PN junctions, a whole family of devices emerges: LEDs that convert current into light,
photodiodes that convert light into current, solar cells that harvest energy from the sun,
and laser diodes that produce coherent beams. This notebook explores the physics
connecting all of them.

---
## Concept — Bandgap Energy and Light

Every semiconductor has a **bandgap** — the minimum energy an electron needs to jump
from the valence band to the conduction band. When an electron falls *back* across that
gap, it can release energy as a **photon**.

Planck's relation ties the photon energy to its wavelength:

$$E = h f = \frac{h c}{\lambda}$$

where:
- $E$ — photon energy (joules, or eV)
- $h$ — Planck's constant ($6.626 \times 10^{-34}$ J·s)
- $c$ — speed of light ($3 \times 10^8$ m/s)
- $\lambda$ — wavelength (meters)

Rearranged for practical use with energy in electron-volts:

$$\lambda \;(\text{nm}) = \frac{1240}{E_g \;(\text{eV})}$$

| Material | Bandgap (eV) | Wavelength (nm) | Color |
|----------|-------------|-----------------|-------|
| GaAs | 1.42 | 873 | Infrared |
| AlGaInP | 1.9–2.2 | 565–653 | Red–Yellow |
| GaP:N | 2.26 | 549 | Green |
| InGaN | 2.6–3.0 | 413–477 | Blue–Violet |
| GaN | 3.4 | 365 | UV |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Wavelength vs Bandgap Energy for common LED materials ---

materials = {
    'GaAs (IR)':       1.42,
    'AlGaAs (Red)':    1.9,
    'AlGaInP (Amber)': 2.0,
    'GaP:N (Green)':   2.26,
    'InGaN (Blue)':    2.75,
    'InGaN (Violet)':  3.0,
    'GaN (UV)':        3.4,
}

# Continuous curve
Eg = np.linspace(1.0, 4.0, 300)
wavelength = 1240.0 / Eg  # nm

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(Eg, wavelength, 'k-', linewidth=1.5, label=r'$\lambda = 1240 / E_g$')

# Color mapping for dots
dot_colors = ['#8B0000', '#FF0000', '#FF8C00', '#228B22', '#0000FF', '#8A2BE2', '#4B0082']
for idx, (name, eg) in enumerate(materials.items()):
    wl = 1240.0 / eg
    ax.plot(eg, wl, 'o', color=dot_colors[idx], markersize=10, zorder=5)
    ax.annotate(f'{name}\n{wl:.0f} nm', (eg, wl),
                textcoords='offset points', xytext=(10, 5), fontsize=8)

# Visible spectrum band
ax.axhspan(380, 700, alpha=0.10, color='orange', label='Visible spectrum')

ax.set_xlabel('Bandgap Energy (eV)', fontsize=12)
ax.set_ylabel('Wavelength (nm)', fontsize=12)
ax.set_title('LED Wavelength vs Bandgap Energy', fontsize=14)
ax.legend(fontsize=10)
ax.set_xlim(1.0, 4.0)
ax.set_ylim(250, 1300)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### LEDs Revisited — Why Forward Voltage Hints at Color

In Module 02 we measured LED forward voltages and noticed red LEDs drop about 1.8 V
while blue LEDs drop about 3.0 V. Now we understand **why**: the forward voltage is
closely related to the bandgap energy. The junction has to give electrons enough energy
to cross the bandgap, and larger bandgaps produce shorter-wavelength (bluer) photons.

The approximation is rough but useful:

$$\lambda \approx \frac{1240}{V_f}$$

This is only approximate because:
- Some energy is lost to heat (phonons)
- The actual photon energy can differ slightly from the electrical bandgap
- Series resistance in the die adds voltage that does not contribute to photon energy

In [ ]:
# --- Estimate LED wavelength from forward voltage ---

Vf_values = np.array([1.8, 2.0, 2.1, 2.2, 3.0, 3.2, 3.4])
labels = ['Red', 'Orange', 'Amber', 'Yellow', 'Blue', 'Violet', 'UV']
actual_wavelengths = np.array([630, 600, 590, 580, 470, 420, 380])

estimated_wavelengths = 1240.0 / Vf_values

fig, ax = plt.subplots(figsize=(9, 6))
x = np.arange(len(labels))
width = 0.35

bars1 = ax.bar(x - width/2, actual_wavelengths, width, label='Actual Peak Wavelength',
               color='steelblue', edgecolor='black')
bars2 = ax.bar(x + width/2, estimated_wavelengths, width, label=r'Estimated ($1240/V_f$)',
               color='coral', edgecolor='black')

ax.set_xlabel('LED Color', fontsize=12)
ax.set_ylabel('Wavelength (nm)', fontsize=12)
ax.set_title('LED Wavelength: Actual vs Estimated from Forward Voltage', fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(fontsize=10)
ax.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print('LED Color  | Vf (V) | Actual (nm) | Estimated (nm) | Error')
print('-' * 65)
for lbl, vf, actual, est in zip(labels, Vf_values, actual_wavelengths, estimated_wavelengths):
    err = est - actual
    print(f'{lbl:10s} | {vf:.1f}   | {actual:11.0f} | {est:14.0f} | {err:+.0f} nm')

### LED Efficiency and the Blue LED Nobel Prize

**Luminous efficacy** measures how efficiently an LED converts electrical power into
visible light, measured in lumens per watt (lm/W). Modern white LEDs achieve
150–200 lm/W, far exceeding incandescent bulbs (~15 lm/W).

The 2014 Nobel Prize in Physics was awarded to **Isamu Akasaki**, **Hiroshi Amano**, and
**Shuji Nakamura** for the invention of efficient **blue LEDs** using InGaN. Why was this so
important?

- Red and green LEDs existed since the 1960s–70s
- Blue was the missing piece for **white light** (blue LED + phosphor coating)
- GaN was extremely difficult to grow as high-quality crystals
- Achieving p-type doping in GaN took decades of materials science work
- Once blue LEDs worked, white LED lighting became possible, transforming
  energy consumption worldwide

---
## The Material Science Why — Photodiodes

A **photodiode** is the reverse process of an LED. Instead of injecting current to produce
photons, we let photons generate current.

When a photon with energy $\geq E_g$ strikes the depletion region of a PN junction:

1. The photon's energy promotes an electron from valence to conduction band,
   creating an **electron-hole pair**
2. The built-in electric field of the depletion region sweeps electrons toward
   the N side and holes toward the P side
3. This separation of charge creates a **photocurrent**

### Two operating modes:

| Mode | Bias | Speed | Linearity | Use case |
|------|------|-------|-----------|----------|
| **Photovoltaic** | Zero bias (or slight forward) | Slower | Good at low light | Solar cells, light meters |
| **Photoconductive** | Reverse bias | Faster | Excellent over wide range | High-speed receivers, sensors |

In **photoconductive mode**, the reverse bias widens the depletion region, making it easier
to capture photons, and the strong electric field sweeps carriers out quickly — giving
faster response times.

### Solar Cells — Large-Area Photodiodes

A **solar cell** is a photodiode optimized for power generation rather than signal detection:

- Large junction area to capture as many photons as possible
- Anti-reflection coating to minimize wasted light
- Optimized bandgap (~1.1 eV for silicon) to capture the solar spectrum
- Operates in **photovoltaic mode** (zero external bias — it generates voltage)

The IV curve of a solar cell looks like a diode curve shifted down by the photocurrent.
The **maximum power point (MPP)** is where $V \times I$ is largest — this is where solar
charge controllers try to operate.

In [ ]:
# --- Solar Cell IV Curve and Maximum Power Point ---

# Simplified single-diode model for a silicon solar cell
q = 1.602e-19      # electron charge (C)
k = 1.381e-23      # Boltzmann constant (J/K)
T = 300             # temperature (K)
Vt = k * T / q      # thermal voltage ~26 mV

I_ph = 5.0          # photocurrent (A) — a typical ~100 cm^2 cell in full sun
I_0 = 1e-10         # reverse saturation current (A)
n = 1.3             # ideality factor

V = np.linspace(0, 0.65, 500)
I = I_ph - I_0 * (np.exp(V / (n * Vt)) - 1)
I = np.maximum(I, 0)  # clip to zero

P = V * I  # power

# Find MPP
mpp_idx = np.argmax(P)
V_mpp, I_mpp, P_mpp = V[mpp_idx], I[mpp_idx], P[mpp_idx]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# IV curve
ax1.plot(V, I, 'b-', linewidth=2)
ax1.plot(V_mpp, I_mpp, 'ro', markersize=12, label=f'MPP ({V_mpp:.2f} V, {I_mpp:.2f} A)')
ax1.axhline(y=I_ph, color='gray', linestyle='--', alpha=0.5, label=f'$I_{{sc}}$ = {I_ph:.1f} A')
ax1.axvline(x=V[np.argmin(np.abs(I - 0.01))], color='gray', linestyle=':', alpha=0.5)
ax1.set_xlabel('Voltage (V)', fontsize=12)
ax1.set_ylabel('Current (A)', fontsize=12)
ax1.set_title('Solar Cell IV Curve', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 0.7)
ax1.set_ylim(0, 5.5)

# Power curve
ax2.plot(V, P, 'r-', linewidth=2)
ax2.plot(V_mpp, P_mpp, 'ro', markersize=12, label=f'MPP = {P_mpp:.2f} W')
ax2.fill_between(V, P, alpha=0.15, color='red')
ax2.set_xlabel('Voltage (V)', fontsize=12)
ax2.set_ylabel('Power (W)', fontsize=12)
ax2.set_title('Solar Cell Power Curve', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 0.7)

plt.tight_layout()
plt.show()

Voc = V[np.argmin(np.abs(I - 0.0))]
FF = P_mpp / (Voc * I_ph)
print(f'Open-circuit voltage (Voc):  {Voc:.3f} V')
print(f'Short-circuit current (Isc): {I_ph:.2f} A')
print(f'Maximum power point:         {P_mpp:.2f} W at {V_mpp:.3f} V, {I_mpp:.2f} A')
print(f'Fill factor:                 {FF:.3f}')

In [ ]:
# --- Interactive: Photodiode current vs light intensity ---

import ipywidgets as widgets
from IPython.display import display

def plot_photodiode(irradiance_pct=100, reverse_bias_V=5.0):
    """
    Simple photodiode model: photocurrent proportional to irradiance,
    operating in photoconductive (reverse-biased) mode.
    """
    irradiance = irradiance_pct / 100.0  # fraction of full sun

    # Responsivity ~ 0.5 A/W at 900 nm for a Si photodiode
    # Assume 1 cm^2 area, 1000 W/m^2 full sun -> 0.1 W on detector
    I_ph_max = 0.05  # A at full irradiance (50 mA)
    I_dark = 10e-9   # dark current (10 nA)

    V = np.linspace(-reverse_bias_V, 0.6, 500)
    Vt = 0.026
    I_0 = 1e-9

    I_photo = irradiance * I_ph_max
    I = I_0 * (np.exp(V / (1.5 * Vt)) - 1) - I_photo

    fig, ax = plt.subplots(figsize=(9, 5))
    ax.plot(V, I * 1000, 'b-', linewidth=2)
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.axvline(x=0, color='black', linewidth=0.5)

    # Mark operating point in photoconductive mode
    I_at_bias = I_0 * (np.exp(-reverse_bias_V / (1.5 * Vt)) - 1) - I_photo
    ax.plot(-reverse_bias_V, I_at_bias * 1000, 'ro', markersize=10,
            label=f'Photoconductive: {I_at_bias*1000:.2f} mA')

    ax.set_xlabel('Voltage (V)', fontsize=12)
    ax.set_ylabel('Current (mA)', fontsize=12)
    ax.set_title(f'Photodiode IV — Irradiance {irradiance_pct}%, Bias {reverse_bias_V} V', fontsize=13)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-60, 10)
    plt.tight_layout()
    plt.show()

widgets.interact(
    plot_photodiode,
    irradiance_pct=widgets.IntSlider(value=100, min=0, max=100, step=5,
                                      description='Light %'),
    reverse_bias_V=widgets.FloatSlider(value=5.0, min=0.0, max=20.0, step=0.5,
                                        description='Rev Bias (V)')
);

### Laser Diodes and Optocouplers

**Laser diodes** are LEDs taken to the extreme. Above a **threshold current**, the photons
in the junction cavity stimulate other electrons to emit photons *in phase*, producing
**coherent, monochromatic** light. Below threshold, it behaves like an LED. Above
threshold, optical power increases rapidly.

Key laser diode parameters:
- **Threshold current** ($I_{th}$) — current where lasing begins
- **Slope efficiency** — optical power per mA above threshold
- **Wavelength** — very narrow linewidth compared to LEDs

**Optocouplers** combine an LED and a photodetector (photodiode or phototransistor)
in a single package. The light path provides **galvanic isolation** — no electrical
connection between input and output. This is essential for:
- Isolating high-voltage circuits from low-voltage logic
- Breaking ground loops
- Safety isolation in medical and industrial equipment

**IR LEDs and phototransistors** are the workhorses of remote controls, optical encoders,
and proximity sensors. The IR wavelength (~940 nm) is invisible but efficiently detected
by silicon photodetectors.

---
## Experiment — LED Forward Voltage vs Color

### Equipment
- Breadboard and jumper wires
- Red, green, blue, and white LEDs
- 330 ohm resistor (current limiter)
- Bench power supply set to 5V
- Klein MM300 multimeter
- CdS photoresistor (for Experiment 3)

### Experiment 1: Measure Forward Voltages

```
  +5V
   │
   ┌───┐
   │330│  (current-limiting resistor)
   │ Ω │
   └─┬─┘
     │
     ▼  LED (anode)
    ─┬─
     │  LED (cathode)
     │
    GND

  Multimeter: DC Voltage across the LED
  (positive probe on anode, negative on cathode)
```

**Procedure:**
1. Insert LED and 330 ohm resistor in series on breadboard
2. Connect to 5V supply
3. Set Klein MM300 to DC Voltage
4. Measure voltage across each LED
5. Record values and compute estimated wavelength using $\lambda = 1240/V_f$

**Expected results:**

| LED | Expected $V_f$ | Estimated $\lambda$ |
|-----|---------------|--------------------|
| Red | ~1.8–2.0 V | ~620–690 nm |
| Green | ~2.0–2.2 V | ~560–620 nm |
| Blue | ~2.9–3.2 V | ~390–430 nm |
| White | ~2.8–3.2 V | (phosphor, not single wavelength) |

### Experiment 2: LED as a Crude Photodiode

An LED *is* a PN junction — so it can also generate a voltage from light.

```
  Bright light source (flashlight / sunlight)
        │
        ▼
    ┌───────┐
    │  LED  │  (NO power supply — disconnected)
    └──┬──┬─┘
       │  │
       V+ V-   Multimeter set to DC mV
```

**Procedure:**
1. Remove the LED from the circuit (no power)
2. Set Klein MM300 to DC millivolts
3. Connect probes across the LED leads
4. Shine a bright light (phone flashlight) directly at the LED
5. Observe the small voltage generated

**Expected:** You should see 100–500 mV depending on LED type and light intensity.
Clear-lens LEDs work best. This is the photovoltaic effect in action.

### Experiment 3: CdS Photoresistor — Light vs Dark

```
  +5V
   │
   ┌───────┐
   │  CdS  │  (photoresistor)
   │  LDR  │
   └──┬────┘
      │
      ├──── V_out ──── Multimeter (DC Volts)
      │
   ┌──┴──┐
   │ 10k │  (fixed resistor)
   └──┬──┘
      │
     GND
```

**Procedure:**
1. Build voltage divider with CdS photoresistor on top, 10k resistor on bottom
2. Measure V_out with multimeter
3. Cover the photoresistor with your hand — note the voltage
4. Shine a flashlight on it — note the voltage

**Expected:**
- Dark: CdS resistance ~200k–1M ohm, V_out close to 0 V
- Room light: CdS ~5k–20k ohm, V_out ~1.5–3.5 V
- Bright light: CdS ~100–500 ohm, V_out close to 5 V

Measure the resistance directly too (disconnect from circuit, set multimeter to ohms).

---
## Simulation — Build It in Falstad

### LED Current Limiting

**What to build:**
1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Place an LED: right-click → **Draw** → **Active Components** → **Add LED**
3. Add a current-limiting resistor (220Ω) in series, powered by a 5V DC source
4. Add ground to complete the loop

**What to observe:**
- Hover over the LED — note the forward voltage (~2V) and forward current
- Change the resistor value: lower resistance = more current = brighter LED (in the simulation, the LED graphic gets brighter)
- Remove the resistor entirely and watch the current spike to a destructive level — this is why current limiting is mandatory

### Photodiode Model

**What to build:**
1. Open a [new blank canvas](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. To model a photodiode: place a current source (representing photocurrent) in parallel with a diode, both connected to a load resistor
3. The current source represents light hitting the photodiode — more current = more light

**What to observe:**
- The voltage across the load resistor is proportional to the photocurrent (light intensity)
- This is the basic operating principle of photodiodes, solar cells, and optical sensors

For a **solar cell**, the model is the same but with larger photocurrents. Add a small shunt resistance in parallel and a small series resistance to make it more realistic.

---
## Datasheet Connection

When you look at an LED datasheet, here is where today's concepts appear:

| Datasheet Parameter | What It Means | Our Concept |
|--------------------|--------------|-----------|
| **Dominant Wavelength** | Peak emission wavelength | $\lambda = 1240/E_g$ |
| **Forward Voltage (V_f)** | Voltage drop at rated current | Related to bandgap |
| **Luminous Intensity (mcd)** | Brightness at rated current | Efficiency |
| **Viewing Angle** | Half-power beam width | Lens optics |
| **Radiant Power** | Total optical output in mW | Efficiency |

For **photodiodes**:
- **Responsivity (A/W)** — current generated per watt of light
- **Dark current** — leakage when no light is present
- **Spectral response curve** — which wavelengths it detects
- **Rise/fall time** — speed of response
- **NEP (Noise Equivalent Power)** — minimum detectable light

For **solar cells**:
- **Voc** (open circuit voltage), **Isc** (short circuit current)
- **Fill factor** — how "square" the IV curve is (ideal = 1.0)
- **Efficiency** — electrical power out / solar power in

---
## Checkpoint Questions

Test your understanding before moving on.

1. **Why does a blue LED have a higher forward voltage than a red LED?**

2. **A LED has a bandgap of 2.0 eV. What wavelength of light does it emit?
   What color would you see?**

3. **What is the difference between photovoltaic and photoconductive mode
   for a photodiode? Which is faster?**

4. **Why does a solar cell have a "maximum power point" rather than simply
   generating maximum power at maximum voltage or maximum current?**

5. **You measure a CdS photoresistor at 500 ohms in bright light and
   200 kohms in the dark. In a voltage divider with a 10k bottom resistor
   and 5V supply, what is V_out in each case?**

6. **Why are optocouplers used instead of just connecting two circuits with a wire?**

7. **What happens to a laser diode below its threshold current?
   What changes above threshold?**

In [ ]:
# --- Quick calculation helper for checkpoint question 5 ---

Vcc = 5.0
R_bottom = 10_000  # 10k ohm

for condition, R_ldr in [('Bright light', 500), ('Dark', 200_000)]:
    V_out = Vcc * R_bottom / (R_ldr + R_bottom)
    print(f'{condition}: R_LDR = {R_ldr:>7,} ohm  ->  V_out = {V_out:.3f} V')